In [1]:
!pip install mlxtend --user
!pip install lightgbm --user
!pip install xgboost --user
!pip install plotly --user

In [1]:
import os
import pandas as pd
import scipy as sp
import numpy as np
import math
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
# import statsmodels.api as sm
# import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_winter = pd.read_csv('/mnt/ETL/Datasets/CO2_prediction_dataset.csv').drop(columns =['Unnamed: 0'])
print(display(df_winter))

,CO2_kg_per_Ha,Crop_Protection_Application_Doses,Soil_Organic_Matter,N_total_kg_per_Ha,P_total_kg_per_Ha,K_total_kg_per_Ha
0,-220.35,14.0,2.500000,205.5,48.0,48.0
1,415.52,13.0,2.500000,198.0,60.0,60.0
2,724.51,9.0,2.500000,202.4,95.2,96.0
3,502.10,9.0,2.500000,179.4,95.2,90.0
4,539.27,10.0,2.500000,179.4,95.2,102.0
...,...,...,...,...,...,...
368,373.55,19.0,3.333333,208.1,98.8,90.0
369,470.26,21.0,3.258333,208.1,98.8,90.0
370,784.98,22.0,2.893750,210.4,98.8,90.0
371,327.29,21.0,3.181250,170.3,98.8,90.0


None


In [3]:
class PredictorWithScalars:
    
    def __init__(self, predictor, X_scalar, X_transformer, y_scalar, y_transformer):
        self.predictor = predictor
        self.X_scalar = X_scalar
        self.X_transformer = X_transformer
        self.y_scalar = y_scalar
        self.y_transformer = y_transformer
    
    def transform_X(self, X):
        transformed_X = self.X_transformer.transform(self.X_scalar.transform(X))
        return transformed_X
    
    def transform_y(self, y):
        transformed_y = self.y_transformer.transform(self.y_scalar.transform(y))
        return transformed_y

    def predict(self, X):
        transformed_X = self.X_transformer.transform(self.X_scalar.transform(X))
        transformed_y_preds = self.predictor.predict(transformed_X).reshape(-1,1)
        inverse_transformed_y_preds = self.y_scalar.inverse_transform(self.y_transformer.inverse_transform(transformed_y_preds))
        return inverse_transformed_y_preds

In [4]:
def build_model(dataset, outlier_removal_lower_percentile, seed):
    
    dataset = dataset[(dataset['CO2_kg_per_Ha'] > dataset['CO2_kg_per_Ha'].quantile(outlier_removal_lower_percentile)) \
                                                                   & (dataset['CO2_kg_per_Ha'] < dataset['CO2_kg_per_Ha'].quantile(1-outlier_removal_lower_percentile))].reset_index(drop = True)

    train_CO2_kg_per_ha, test_CO2_kg_per_ha = train_test_split(dataset, test_size = 0.18, random_state = seed)

    X_train_CO2_kg_per_ha = train_CO2_kg_per_ha.drop(columns = ['CO2_kg_per_Ha'])
    X_test_CO2_kg_per_ha = test_CO2_kg_per_ha.drop(columns = ['CO2_kg_per_Ha'])

    y_train_CO2_kg_per_ha = np.array(train_CO2_kg_per_ha['CO2_kg_per_Ha']).reshape(-1, 1)
    y_test_CO2_kg_per_ha = np.array(test_CO2_kg_per_ha['CO2_kg_per_Ha']).reshape(-1, 1)
    mean_CO2_kg_per_ha = np.mean(y_train_CO2_kg_per_ha)

    standard_scaler_X = StandardScaler().fit(X_train_CO2_kg_per_ha)
    power_transform_X = PowerTransformer(standardize = True).fit(standard_scaler_X.transform(X_train_CO2_kg_per_ha))

    standard_scaler_y = StandardScaler().fit(y_train_CO2_kg_per_ha)
    power_transform_y = PowerTransformer(standardize = True).fit(standard_scaler_y.transform(y_train_CO2_kg_per_ha))

    xgboost = XGBRegressor(learning_rate = 0.005, n_estimators = 10000, max_depth = 3, min_child_weight = 0, gamma = 0, subsample = 0.7, \
                                              colsample_bytree = 0.7, objective = 'reg:linear', nthread = -1, scale_pos_weight = 1, seed = 27, reg_alpha = 0.00006)

    predictorScaler = PredictorWithScalars(xgboost, standard_scaler_X, power_transform_X, standard_scaler_y, power_transform_y)
    predictorScaler.predictor = predictorScaler.predictor.fit(predictorScaler.transform_X(X_train_CO2_kg_per_ha), predictorScaler.transform_y(y_train_CO2_kg_per_ha))
    
    return predictorScaler

In [5]:
def eval_model(df_winter_combined_parents_final_test_set, model):
    
    final_preds = np.zeros((len(df_winter_combined_parents_final_test_set), 1))
    
    X_df_winter_combined_parents_final_test_set = df_winter_combined_parents_final_test_set.drop(columns = ['CO2_kg_per_Ha'])
    y_df_winter_combined_parents_final_test_set = np.array(df_winter_combined_parents_final_test_set['CO2_kg_per_Ha']).reshape(-1,1)
    final_preds[:,0] = model.predict(X_df_winter_combined_parents_final_test_set)[:,0]
        
    mean_abs_error = mean_absolute_error(y_df_winter_combined_parents_final_test_set, final_preds)
    root_mean_sq_error = np.sqrt(mean_squared_error(y_df_winter_combined_parents_final_test_set, final_preds))
    r_sq = r2_score(y_df_winter_combined_parents_final_test_set, final_preds)
    
    return mean_abs_error, root_mean_sq_error, r_sq

In [7]:
import pickle
import random
# from sklearn.externals import joblib

random.seed(10)
random_state_list1 = random.sample(range(1000), 5)

random.seed(11)
random_state_list2 = random.sample(range(1000), 5)

df_winter_combined_parents_temp = df_winter.sample(frac = 1, random_state = random_state_list1[0]).reset_index(drop = True)
df_winter_combined_parents_final_test_set = df_winter_combined_parents_temp[:57]
df_winter_combined_parents_temp = df_winter_combined_parents_temp[57:].reset_index(drop = True)

predictor_scaler = build_model(df_winter_combined_parents_temp, 0.05, random_state_list2[0])
pickle.dump(predictor_scaler.X_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Scalar_CO2.pkl', 'wb'))
pickle.dump(predictor_scaler.y_scalar, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_Scalar_CO2.pkl', 'wb'))
pickle.dump(predictor_scaler.X_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/X_Transformer_CO2.pkl', 'wb'))
pickle.dump(predictor_scaler.y_transformer, open('/mnt/ETL/pkl_predictors_scalars_transformers/y_transformer_CO2.pkl', "wb"))
pickle.dump(predictor_scaler.predictor, open("/mnt/ETL/pkl_predictors_scalars_transformers/predictor_CO2.pkl", "wb"))

[16:20:58] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
import random
random.seed(10)
random_state_list1 = random.sample(range(1000), 5)

random.seed(11)
random_state_list2 = random.sample(range(1000), 5)

df_winter_combined_parents_temp = df_winter.sample(frac = 1, random_state = random_state_list1[0]).reset_index(drop = True)
df_winter_combined_parents_final_test_set = df_winter_combined_parents_temp[:57]
df_winter_combined_parents_temp = df_winter_combined_parents_temp[57:].reset_index(drop = True)

predictor_scaler = build_model(df_winter_combined_parents_temp, 0.05, random_state_list2[0])

eval_model_temp = eval_model(df_winter_combined_parents_temp, predictor_scaler)
print(eval_model_temp[0], eval_model_temp[1], eval_model_temp[2])

[16:21:53] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
